In [11]:
%pip uninstall mysql-python
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 5.5 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import uvicorn
from typing import List
from fastapi import FastAPI
import mysql.connector
from dotenv import load_dotenv
from fastapi.middleware.cors import CORSMiddleware
from abc import ABC, abstractmethod
load_dotenv()

True

In [10]:
import os
import mysql.connector
from abc import ABC, abstractmethod

class Database(ABC):
    """
    Database context manager
    """

    def __init__(self, driver) -> None:
        self.driver = driver

    @abstractmethod
    def connect_to_database(self):
        raise NotImplementedError()

    def __enter__(self):
        self.connection = self.connect_to_database()
        self.cursor = self.connection.cursor()
        return self

    def __exit__(self, exception_type, exc_val, traceback):
        self.cursor.close()
        self.connection.close()

# Access environment variables
db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_database = os.getenv("DB_DATABASE")
db_port = os.getenv("DB_PORT")

class MysqlDatabase(Database):
    """MySQL Database context manager"""

    def __init__(self) -> None:
        self.driver = mysql.connector
        super().__init__(self.driver)

    def connect_to_database(self):
        config = {
            'user': db_user,
            'password': db_password,
            'host': db_host,
            'port': db_port,
            'database': db_database,
            'raise_on_warnings': True,
            'autocommit': False  # Disable autocommit to manage transactions manually
        }
        return self.driver.connect(**config)

if __name__ == "__main__":
    # Create a MysqlDatabase object within a with block
    with MysqlDatabase() as db:
        # Execute a simple query
        db.cursor.execute("SELECT VERSION()")
        # Fetch the result
        version = db.cursor.fetchone()[0]
        print(f"MySQL Database version: {version}")


MySQL Database version: 8.3.0


In [2]:
import mysql.connector
import os
# Establish connection parameters
host = os.environ.get("DB_HOST")
user = os.environ.get("DB_USERNAME")
password = os.environ.get("DB_PASSWORD")
database = os.environ.get("DB_DATABASE")

# Connect to MySQL
try:
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password="Geology@7",
        database=database
    )
    if connection.is_connected():
        print("Connected to MySQL database")
        # Perform your database operations here
    else:
        print("Connection failed")
except Exception as e:
    print("Error connecting to MySQL:", e)
finally:
    # Close connection
    if 'connection' in locals():
        connection.close()
        print("MySQL connection closed")


Connected to MySQL database
MySQL connection closed


In [4]:
import os
import uvicorn
from typing import List
from fastapi import FastAPI, Depends
import mysql.connector

try:
    # Get MySQL connection details from environment variables
    host = os.environ.get("DB_HOST")
    user = os.environ.get("DB_USERNAME")
    password = os.environ.get("DB_PASSWORD")
    database = os.environ.get("DB_DATABASE")
    # mysql_config = {
    #     'user': user,
    #     'password': password,
    #     'host': ,
    #     'database': database,
    #     'port': '3306',
    #     'ssl_disabled': True
    # }
    db = mysql.connector.connect(
        host=host,
        user=user,
        port='3306',
        password="Geology@7",
        database=database,
        use_pure=True
    )
    print("Connected to MySQL successfully!")

except mysql.connector.Error as err:
    print("Error:", err)

Error: 1045 (28000): Access denied for user ''@'localhost' (using password: YES)
